In [3]:
import findspark
findspark.init("/home/raj/spark/")

In [6]:
from pyspark.ml.clustering import KMeans
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer , VectorAssembler
#creation of spark session


def K_means():
    
    spark = SparkSession \
        .builder \
        .appName("Python Spark ML example") \
        .getOrCreate()
# Loads data.
    dataset= spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("/home/raj/Downloads/notenook/adult2.csv")



#creation of the features vector and transforming the dataset for the same
    workclass_stringIndexer = StringIndexer(inputCol="workclass", outputCol="workclass_indexed")
    si_model_workclass = workclass_stringIndexer.fit(dataset)
    education_stringIndexer = StringIndexer(inputCol="education", outputCol="education_indexed")
    si_model_education = education_stringIndexer.fit(dataset)
    occupation_stringIndexer = StringIndexer(inputCol="occupation", outputCol="occupation_indexed")
    si_model_occupation = occupation_stringIndexer.fit(dataset)
    sex_stringIndexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
    si_model_sex = sex_stringIndexer.fit(dataset)
    income_stringIndexer = StringIndexer(inputCol="income", outputCol="income_indexed")
    si_model_income = income_stringIndexer.fit(dataset)
    td = si_model_workclass.transform(si_model_education.transform(si_model_occupation.transform(si_model_sex.transform(si_model_income.transform(dataset)))))
    td.toPandas()


#selecting the required columns
    feature_columns = ["age","workclass_indexed","fnlwgt","education_num","hours_per_week","sex_indexed","education_indexed","occupation_indexed","income_indexed"]

    feature_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

    td_dataset_prepared = feature_assembler.transform(td)
    td_dataset_prepared.printSchema()
    td_dataset_prepared.toPandas()


#making the k_means model
    new1=td_dataset_prepared.select("features")
#new1.show()
    kmeans = KMeans().setK(2).setSeed(1)
    model = kmeans.fit(new1)
    transformed = model.transform(td_dataset_prepared)
    transformed.toPandas()


#calculation of SSE
    wssse = model.computeCost(td_dataset_prepared)
    print("Within Set Sum of Squared Errors = " + str(wssse))


#Computing the final clusters of the dataset
    centers = model.clusterCenters()
    print("Cluster Centers: ")
    for center in centers:
        print(center)
        
        return center
    

In [7]:
K_means()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- income_indexed: double (nullable = true)
 |-- sex_indexed: double (nullable = true)
 |-- occupation_indexed: double (nullable = true)
 |-- education_indexed: double (nullable = true)
 |-- workclass_indexed: double (nullable = true)
 |-- features: vector (nullable = true)

Within Set Sum of Squared Errors = 13532238195992.078
Cluster Centers: 
[3.72389937e+01 8.69182

array([3.72389937e+01, 8.69182390e-01, 3.23380658e+05, 9.88679245e+00,
       3.98993711e+01, 3.03144654e-01, 2.55974843e+00, 4.34842767e+00,
       2.13836478e-01])